### Zadanie: Przewidywanie Etykiet Ze Zbioru Reuters

Twoim zadaniem jest zbudowanie modelu klasyfikacji tekstu, który będzie przewidywał etykiety tematów dla dokumentów ze zbioru Reuters. Zbiór Reuters jest popularnym zbiorem danych do zadania klasyfikacji wieloklasowej, gdzie każdy dokument jest przypisany do jednej lub kilku etykiet tematów.

**Opis Zadania**

Twoim celem jest zbudowanie modelu klasyfikacji, który nauczy się na podstawie opisów dokumentów z zestawu Reuters i przewidzi, do jakich etykiet tematów każdy dokument należy. W tym przypadku, zadanie klasyfikacji jest problemem wieloklasowym, ponieważ każdy dokument może być przypisany do jednej lub wielu etykiet tematów.

**Instrukcje**

1. Zaimportuj odpowiednie moduły z biblioteki Keras i przygotuj dane ze zbioru Reuters.

2. Zdefiniuj architekturę modelu klasyfikacji. Możesz użyć różnych warstw, takich jak Embedding, LSTM, Dense itp., aby zbudować model.

3. Podziel dane na zbiór treningowy i zbiór testowy w celu oceny jakości modelu.

4. Skonfiguruj model, wybierz odpowiedni optymalizator i funkcję straty.

5. Trenuj model na danych treningowych.

6. Oceń wyniki treningu i przetestuj działanie modelu na danych testowych.

**Wskazówki**

- Upewnij się, że odpowiednio przetworzyłeś dane tekstowe ze zbioru Reuters przed użyciem ich do treningu modelu. Możesz użyć technik tokenizacji, wektoryzacji lub kodowania one-hot.

- W zadaniu klasyfikacji wieloklasowej, odpowiednia funkcja straty to `sparse_categorical_crossentropy` lub `categorical_crossentropy`, w zależności od tego, jak zorganizowane są etykiety w danych.

- Eksperymentuj z różnymi architekturami modelu, rozmiarami warstw i hiperparametrami, aby uzyskać jak najlepszą wydajność klasyfikacji.

- Monitoruj wyniki na zbiorze walidacyjnym, aby uniknąć nadmiernego dopasowania (overfitting) modelu.



In [1]:
from keras.datasets import reuters
from keras.preprocessing import sequence

# Przygotowanie danych
max_features = 10000
max_len = 200

word_index = reuters.get_word_index()

# Odwrócenie mapowania indeksów słów na słowa
reverse_word_index = dict((index, word) for (word, index) in word_index.items())

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


550378/550378 [==============================] - 0s 0us/step


### Zadanie: Analiza Sentymentu na podstawie Tweetów

Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie klasyfikować tweety na podstawie ich sentymentu. W danych mamy dostarczone tweety oraz etykiety sentymentu (pozytywny, negatywny, neutralny).

Kroki, które musisz podjąć:

1. **Przygotowanie danych**: Dane zostały wczytane z pliku CSV. Teksty tweetów znajdują się w kolumnie "text", a etykiety sentymentu w kolumnie "sentiment". Teksty tweetów oraz etykiety sentymentu są przygotowane do dalszej obróbki.

2. **Tokenizacja tekstu**: Teksty tweetów są tokenizowane za pomocą `Tokenizer`, a następnie przekształcane na sekwencje liczb. Jest to niezbędne, aby można było używać ich jako danych wejściowych do modelu sieci neuronowej.

3. **Model sieci neuronowej**: Model sieci neuronowej jest zdefiniowany w kodzie i składa się z następujących warstw:
   - **Warstwa wejściowa**: Przyjmuje sekwencje tokenów o maksymalnej długości `max_sequence_length`.
   - **Warstwa Embedding**: Mapuje tokeny na wektory o wymiarze 100.
   - **Warstwa LSTM**: Warstwa rekurencyjna LSTM o rozmiarze 64, która przetwarza sekwencje wektorów.
   - **Warstwa Dense**: Warstwa wyjściowa z funkcją aktywacji softmax, która generuje trzy możliwe klasy sentymentu.

4. **Kompilacja modelu**: Model jest kompilowany z optymalizatorem Adam, funkcją straty "sparse_categorical_crossentropy" (odpowiednia do klasyfikacji wieloklasowej) oraz metryką "accuracy".

5. **Trenowanie modelu**: Model jest trenowany na danych tweetów przez 5 epok, z wsadem o rozmiarze 32. Wynik trenowania jest oceniany na zbiorze walidacyjnym.

Twoim zadaniem jest uzupełnienie tego kodu i dostosowanie parametrów modelu oraz trenowania, aby uzyskać jak najlepsze wyniki w zadaniu analizy sentymentu na podstawie tweetów. Możesz eksperymentować z parametrami, takimi jak liczba epok trenowania, rozmiar wsadu (batch size), wymiar embeddingu czy rozmiar warstwy LSTM, aby dostosować model do danego zadania analizy sentymentu tekstu.

In [2]:
import pandas as pd 

df = pd.read_csv('/kaggle/input/twitter-tweets-sentiment-dataset/Tweets.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
def clean_sentence_enhanced_v2(sentence):
   # Konwersja na małe litery
   sentence = sentence.lower()
   # Usuwanie kwot z symbolem dolara i liczb z przecinkami
   sentence = re.sub(r'\$\d+(?:\.\d+)?', '', sentence)
   sentence = re.sub(r'\d+,\d+', '', sentence)
   # Zamiana URLi
   sentence = re.sub(r'http\S+|www\S+|https\S+', '<URL>', sentence, flags=re.MULTILINE)
   # Zamiana wzmianek o użytkownikach
   sentence = re.sub(r'@\w+', '<USER>', sentence)
   # Zamiana znaków '#'
   sentence = re.sub(r'#', '<TAG>', sentence)
   # Zamiana pozostałych cyfr
   sentence = re.sub(r'\d+', '<NUMBER>', sentence)
   # Usuwanie znaków interpunkcyjnych
   sentence = ''.join(ch for ch in sentence if ch not in string.punctuation)
   return sentence

In [5]:
# Przygotuj teksty i etykiety sentymentu
import re
import string
texts = df['text'].astype(str)
sentiment_map = {"positive": 1, "negative": 0, 'neutral': 2}
labels = df['sentiment'].map(sentiment_map).tolist()
texts=texts.apply(clean_sentence_enhanced_v2).tolist()

In [ ]:
labels

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.utils import to_categorical

# Tokenizacja tekstu
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
#y = to_categorical(labels, 3)
y=np.array(labels)

In [7]:
y

array([2, 0, 0, ..., 1, 1, 2])

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(sequences, y, test_size=0.3, random_state=42)


In [14]:
max_sequence_length = max(len(seq) for seq in texts)
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_val = pad_sequences(X_val, maxlen=max_sequence_length)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)

In [20]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,Dropout
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Embedding(input_dim=max_sequence_length, output_dim=100))
model.add(LSTM(64))
model.add(Dense(256))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=5000, verbose=1, validation_data=(X_val,y_val))


Epoch 1/50
4/4 [==============================] - 4s 372ms/step - loss: 1.0928 - accuracy: 0.3917 - val_loss: 1.0827 - val_accuracy: 0.4089
Epoch 2/50
4/4 [==============================] - 1s 285ms/step - loss: 1.0821 - accuracy: 0.4027 - val_loss: 1.0792 - val_accuracy: 0.4089
Epoch 3/50
4/4 [==============================] - 1s 233ms/step - loss: 1.0780 - accuracy: 0.4035 - val_loss: 1.0735 - val_accuracy: 0.4175
Epoch 4/50
4/4 [==============================] - 1s 214ms/step - loss: 1.0713 - accuracy: 0.4206 - val_loss: 1.0639 - val_accuracy: 0.4361
Epoch 5/50
4/4 [==============================] - 1s 232ms/step - loss: 1.0588 - accuracy: 0.4339 - val_loss: 1.0479 - val_accuracy: 0.4462
Epoch 6/50
4/4 [==============================] - 1s 251ms/step - loss: 1.0395 - accuracy: 0.4657 - val_loss: 1.0240 - val_accuracy: 0.4720
Epoch 7/50
4/4 [==============================] - 1s 233ms/step - loss: 1.0094 - accuracy: 0.4918 - val_loss: 0.9884 - val_accuracy: 0.4979
Epoch 8/50
4/4 [====


### Zadanie: Klasyfikacja płci na podstawie obrazów

Twoim zadaniem jest stworzenie modelu sieci neuronowej do klasyfikacji płci (męskiej lub żeńskiej) na podstawie zdjęć. Zadanie to jest typowym zadaniem binarnej klasyfikacji.

Kroki, które musisz podjąć:

1. **Przygotowanie danych**: Dane treningowe i testowe zostały wczytane za pomocą generatorów `train_generator` i `test_generator`. Obrazy zostały już przeskalowane i przydzielone do odpowiednich klas (binary). Twoim zadaniem jest wykorzystanie tych danych do trenowania i testowania modelu.

2. **Model sieci neuronowej**: Model sieci neuronowej nie został jeszcze zdefiniowany. Twoim zadaniem jest utworzenie modelu, który będzie potrafił skutecznie klasyfikować płci na podstawie zdjęć. Możesz użyć architektury ResNet50. Upewnij się, że model ma odpowiednią liczbę neuronów w warstwie wyjściowej (1 neuron dla binary classification) oraz odpowiednią funkcję aktywacji.

3. **Kompilacja modelu**: Model musi zostać skompilowany z odpowiednim optymalizatorem i funkcją straty. W przypadku binarnej klasyfikacji, odpowiednią funkcją straty jest 'binary_crossentropy'. Możesz również monitorować metryki, takie jak 'accuracy'.

4. **Trenowanie modelu**: Trenuj model na danych treningowych z użyciem generatora `train_generator`. Możesz dostosować liczbę epok, rozmiar wsadu (batch size) i inne parametry trenowania. Upewnij się, że model jest trenowany na odpowiedniej liczbie epok, aby uzyskać satysfakcjonujące wyniki.

5. **Ewaluacja modelu**: Po zakończeniu trenowania, ocen model na danych testowych z użyciem generatora `test_generator`. Sprawdź jakość klasyfikacji i inne metryki, aby ocenić skuteczność modelu w zadaniu klasyfikacji płci na podstawie obrazów.

Twoim celem jest stworzenie modelu, który będzie w stanie skutecznie klasyfikować płci na podstawie dostępnych zdjęć i uzyskać jak najlepsze wyniki. Możesz eksperymentować z architekturą modelu, parametrami trenowania i innymi aspektami, aby osiągnąć najlepsze rezultaty.

In [32]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, GlobalAveragePooling2D, AveragePooling2D, Flatten

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ustawienie ścieżki do katalogu z danymi

# Definicja generatora obrazów
datagen = ImageDataGenerator(
    rescale=1.0/255,           # Przeskalowanie wartości pikseli do zakresu [0, 1]
    validation_split=0.2       # Procent danych przeznaczonych na zbiór walidacyjny
)

# Wczytanie danych treningowych
train_generator = datagen.flow_from_directory(
    '/kaggle/input/gender-classification-dataset/Training',
    target_size=(224, 224),    # Rozmiar docelowy obrazów
    batch_size=64,             # Rozmiar wsadu
    class_mode='binary',  # Rodzaj klasyfikacji (binary lub categorical)
    subset='training'          # Ustawienie dla danych treningowych
)


# Wczytanie danych treningowych
test_generator = datagen.flow_from_directory(
    '/kaggle/input/gender-classification-dataset/Validation',
    target_size=(224, 224),    # Rozmiar docelowy obrazów
    batch_size=64,             # Rozmiar wsadu
    class_mode='binary',  # Rodzaj klasyfikacji (binary lub categorical)
    subset='validation'          # Ustawienie dla danych treningowych
)


Found 37608 images belonging to 2 classes.
Found 2329 images belonging to 2 classes.


In [36]:
# Sprawdź liczbę klas
num_classes = len(train_generator.class_indices)
print("Liczba klas:", num_classes)

# Sprawdź rozmiar pierwszej partii danych treningowych
batch_images, batch_labels = train_generator.next()
print("Rozmiar partii obrazów:", batch_images.shape)
print("Rozmiar partii etykiet:", batch_labels.shape)

Liczba klas: 2
Rozmiar partii obrazów: (64, 224, 224, 3)
Rozmiar partii etykiet: (64,)


In [24]:
female


NameError: name 'female' is not defined

In [41]:

#todo model z ResNET
num_epochs = 1
output_logistic_regression = 1
from tensorflow.keras.applications import ResNet152

base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#for layer in base_model.layers:
 #   layer.trainable = False

# Adding new classification layers
model = Sequential([   base_model,   GlobalAveragePooling2D(),   Dense(output_logistic_regression, activation='sigmoid')])
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=5, verbose=1, validation_data=test_generator)


Epoch 1/5
588/588 [==============================] - 1055s 2s/step - loss: 0.6934 - accuracy: 0.5876 - val_loss: 0.8129 - val_accuracy: 0.5354
Epoch 2/5
588/588 [==============================] - 965s 2s/step - loss: 0.2710 - accuracy: 0.8868 - val_loss: 0.2544 - val_accuracy: 0.8987
Epoch 3/5
588/588 [==============================] - 963s 2s/step - loss: 0.1642 - accuracy: 0.9392 - val_loss: 0.9942 - val_accuracy: 0.5479
Epoch 4/5
588/588 [==============================] - 964s 2s/step - loss: 0.1419 - accuracy: 0.9474 - val_loss: 0.1771 - val_accuracy: 0.9300
Epoch 5/5
588/588 [==============================] - 965s 2s/step - loss: 0.1306 - accuracy: 0.9518 - val_loss: 0.2178 - val_accuracy: 0.9257


In [45]:
base_model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_13[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                          

In [ ]:
#jeśli zamrażamy ResNet to niezależnie od wersji (50,102,152) mamy dokładność na max 70%. Jeżeli odmrozimy warstwy ResNet to mamy dokładność nawet 95%. Do zbadania w przyszłości co jeżeli odmrozimy tylko ostatnie warstwy ResNet